This is a first pass exploration of text preprocessing using SpaCy and topic modeling with Genism's LDA algorithm. 

In [2]:
import pandas as pd
import os
data_file_path = os.path.join('..', 'SatisfAI-data', 'randomsample500.csv')
out_dir = os.path.join('..', 'SatisfAI-data', 'Dec_data')
randomsample500 = pd.read_csv(data_file_path)
randomsample500.head()

,tickets_id,tickets_assignee_id,tickets_created_at,tickets_custom_field_20180577,tickets_custom_field_21024293,tickets_custom_field_22526998,tickets_custom_field_24112003,tickets_custom_field_27311858,tickets_custom_field_505895,tickets_description,...,metric_sets_reply_time_in_minutes_business,metric_sets_reply_time_in_minutes_calendar,metric_sets_requester_updated_at,metric_sets_requester_wait_time_in_minutes_business,metric_sets_requester_wait_time_in_minutes_calendar,metric_sets_solved_at,metric_sets_status_updated_at,metric_sets_updated_at,ticket_channel,lob
0,52814760,14143076948,2016-12-14 20:24:01.000000,622331045.0,refund_issued,reason4_2g13,ctlamkl@yahoo.com,LG-TNVK-M1FC-JX4T-R2FC,gg-xbox-360-camouflage-wireless-controller,\nCustomer Contact Reason Drop-down1: Item del...,...,401,557,2016-12-14 20:24:01.000000,401,557,2016-12-15 05:40:48.000000,2016-12-29 06:09:00.000000,2016-12-15 05:40:48.000000,Contact Us,Goods
1,52412687,3958975757,2016-12-08 08:02:06.000000,608726425.0,refund_not_discussed,reason4_2f1,rescuerenee@gmail.com,LG-YWSX-72ZV-7H74-KPKH,wow-starbucks-6,\nCustomer Contact Reason Drop-down1: Managing...,...,0,4,2016-12-08 08:25:05.000000,0,8,2016-12-15 09:04:09.000000,2016-12-29 10:07:29.000000,2016-12-15 09:04:10.000000,Contact Us,Local
2,51805897,1180628008,2016-11-29 04:24:40.000000,255136503.0,refund_issued,reason4_2f3,d_castrosr@yahoo.com,241897357-0-1,mymms-com-9-boston,\nCustomer Contact Reason Drop-down1: Problem ...,...,101,101,2016-12-03 07:02:33.000000,1329,3324,2016-12-02 01:27:38.000000,2016-12-16 02:09:57.000000,2016-12-02 01:27:39.000000,Contact Us,Local
3,52297843,6071705957,2016-12-07 01:13:59.000000,619034955.0,refund_not_discussed,reason4_2g19,vkuhn@nycap.rr.com,GG-19N2-7XKL-32XN-N759,gg-tulipmist-ultrasonic-essential-oil-diffuser,\nCustomer Contact Reason Drop-down1: Item del...,...,406,525,2016-12-07 01:13:58.000000,406,525,2016-12-07 09:58:42.000000,2016-12-21 10:06:38.000000,2016-12-07 09:58:43.000000,Contact Us,Goods
4,52424023,14676246228,2016-12-08 11:03:36.000000,628595395.0,refund_issued,reason4_2f10,snewtonrobbins@gmail.com,LG-7VRW-36BM-WC29-BSTM,shari-s-berries-1103-orange-county,\nCustomer Contact Reason Drop-down1: Somethin...,...,0,477,2016-12-09 23:12:05.000000,0,477,2016-12-08 19:00:26.000000,2016-12-22 20:03:46.000000,2016-12-08 19:00:26.000000,Contact Us,Local


In [3]:
import spacy
nlp = spacy.load('en')

In [17]:
text =randomsample500["tickets_description"][0]

In [21]:
text

"\nCustomer Contact Reason Drop-down1: Item delivery and returns\n\nCustomer Contact Reason Drop-down2: I want to return the item I received\n\nIssue:  I want to edit or cancel my order\n\n\n Please give us a quick idea of what's going on: : Since I ordered this, there has been no activity in regards to shipping so I would like to cancel my order and get my money back\n User Name :AnnMarie Lynch \n Order: https://cs.groupondev.com/users/ann-marie-lynch-12/orders/622331045"

In [27]:
#remove the drop down menu text, user name, and order
import re

dd1 = re.search("(Customer Contact Reason Drop-down1:.*\n)", text)
if dd1:
    text = text[dd1.end():]
dd2 = re.search("(Customer Contact Reason Drop-down2:.*\n)", text)
if dd2:
    text = text[dd2.end():]
issue = re.search("Please give us a quick idea of what's going on:", text)
if issue:
    text = text[issue.end():]
user = re.search("User Name.*\n", text)
if user:
    text = text[:user.start()]
order = re.search("Order:.*\n", text)
if order:
    text = text[:order.start()]

text

' : Since I ordered this, there has been no activity in regards to shipping so I would like to cancel my order and get my money back\n '

In [28]:
parsed_text = nlp(text)

In [29]:
for num, sentence in enumerate(parsed_text.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
 : Since I ordered this, there has been no activity in regards to shipping so I would like to cancel my order and get my money back
 



In [33]:
#normalize text, remove stopwords and punctuation
token_lemma = [token.lemma_ for token in parsed_text if not token.is_stop and not token.is_punct and not token.like_num and not token.is_space]
token_lemma

['order', 'activity', 'regard', 'shipping', 'like', 'cancel', 'order', 'money']